# Problem 686
## Powers of Two

$2 ^ 7 = 128$ is the first power of two whose leading digits are $12$. The next power of two whose leading digits are $12$ is $2 ^ {80}$.

Define $p(L, n)$ to be the $n$th-smallest value of $j$ such that the base $10$ representation of $2 ^ j$ begins with the digits of $L$. So $p(12, 1) = 7$ and $p(12, 2) = 80$.

You are also given that $p(123, 45) = 12710$

Find $p(123, 678910)$

## Solution

In [1]:
from math import ceil, log, log10, modf

In [2]:
def compute(digits: int, n: int) -> int:
    def mod(k: float) -> float:
        k %= 1
        return k if k < 0.5 else k - 1

    def check(k: float) -> bool:
        return (lower and lower_limit <= k < upper_limit) or (not lower and lower_limit >= k > upper_limit)

    limit = (log(digits + 1) - log(digits)) / log(10)
    lower_limit, upper_limit = log(digits) / log(10) % 1, log(digits + 1) / log(10) % 1
    lower = lower_limit < upper_limit
    error = log(2) / log(10)
    x = error % 1
    t, convergent, convergent_error = 0, 1, 1
    while abs(convergent_error) > limit:
        x, integer = modf(1 / x)
        convergent, t = convergent * integer + t, convergent
        convergent_error = mod(convergent * error)
    x, integer = modf(1 / x)
    semi_convergent, semi_convergent_error = t, 1
    while abs(semi_convergent_error) > limit:
        semi_convergent += convergent
        semi_convergent_error = mod(semi_convergent * error)
    differences = (convergent, semi_convergent, convergent + semi_convergent)
    if convergent_error > 0:
        convergent_limit = convergent * ceil(lower_limit / convergent_error)
    else:
        convergent_limit = semi_convergent * ceil(lower_limit / semi_convergent_error)
    convergents = list()
    while convergent < convergent_limit:
        convergents.append(convergent)
        convergent, t = integer * convergent + t, convergent
        x, integer = modf(1 / x)
    convergents = convergents[1:]
    for difference in reversed(convergents):
        while convergent_limit > difference:
            x = (convergent_limit - difference) * error % 1
            if check(x):
                convergent_limit -= difference
            else:
                break
    while True:
        for difference in differences:
            if difference > convergent_limit:
                continue
            x = (convergent_limit - difference) * error % 1
            if check(x):
                convergent_limit -= difference
                break
        else:
            break
    while True:
        for difference in differences:
            x = (convergent_limit + difference) * error % 1
            if check(x):
                n -= 1
                if n == 0:
                    return int(convergent_limit)
                convergent_limit += difference
                break

In [3]:
compute(12, 1)

7

In [4]:
compute(12, 2)

80

In [5]:
compute(123, 45)

12710

In [6]:
compute(123, 678910)

193060223

In [7]:
%timeit -n 100 -r 1 -p 6 compute(123, 678910)

280.494 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)
